In [1]:
from scrape import extract_players
from scrape import extract_year
from scrape import make_dists
from scrape import extract_age
import numpy as np
from sklearn.cluster import KMeans
import pandas as pd
import re
import matplotlib.pyplot as plt
from scrape import read_targets
from scrape import clean_name
from sklearn.svm import SVR
from sklearn.preprocessing import scale
from scrape import svr_2023
from scrape import run_svr_2023
from scrape import run_xgb_2023
from scrape import xgb_2023
from scrape import get_data

In [2]:
results = run_svr_2023('rb', 'PPR', pca=True)
results['projections'].head(50)

austin-ekeler
christian-mccaffrey
josh-jacobs
derrick-henry
saquon-barkley
nick-chubb
rhamondre-stevenson
tony-pollard
aaron-jones
joe-mixon
dalvin-cook
leonard-fournette
jamaal-williams
najee-harris-rb
miles-sanders
alvin-kamara
travis-etienne
kenneth-walker-rb
james-conner
jerick-mckinnon
dandre-swift
ezekiel-elliott
devin-singletary
david-montgomery
raheem-mostert
aj-dillon
dameon-pierce
antonio-gibson
tyler-allgeier
jeffery-wilson
cordarrelle-patterson
latavius-murray
jonathan-taylor
samaje-perine
cam-akers
rachaad-white
isiah-pacheco
donta-foreman
kareem-hunt
michael-carter-rb
khalil-herbert
breece-hall
brian-robinson-jr
james-cook
kenyan-drake
clyde-edwards-helaire
jaylen-warren
damien-harris-rb
eno-benjamin
james-robinson
alexander-mattison
kenneth-gainwell
chuba-hubbard
melvin-gordon
deon-jackson
nyheim-hines
jk-dobbins
dontrell-hilliard
chase-edmonds
darrell-henderson
rex-burkhead
jamycal-hasty
joshua-kelley
zack-moss
matt-breida
gus-edwards
zonovan-bam-knight
jd-mckissic
amee

c:\Python\Python39\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Done for 2022
jonathan-taylor
austin-ekeler
najee-harris-rb
joe-mixon
james-conner
leonard-fournette
ezekiel-elliott
alvin-kamara
cordarrelle-patterson
antonio-gibson
aaron-jones
josh-jacobs
nick-chubb
damien-harris-rb
dandre-swift
dalvin-cook
javonte-williams
devin-singletary
darrel-williams
david-montgomery
melvin-gordon
derrick-henry
aj-dillon
james-robinson
myles-gaskin
elijah-mitchell-rb
darrell-henderson
tony-pollard
michael-carter-rb
saquon-barkley
devonta-freeman
sony-michel
devontae-booker
chase-edmonds
mike-davis-rb
chuba-hubbard
jd-mckissic
christian-mccaffrey
alexander-mattison
brandon-bolden
kenneth-gainwell
rashaad-penny
jamaal-williams
miles-sanders
ty-johnson
clyde-edwards-helaire
rhamondre-stevenson
nyheim-hines
kareem-hunt
mark-ingram
zack-moss
rex-burkhead
dernest-johnson
latavius-murray
kenyan-drake
boston-scott
donta-foreman
ameer-abdullah
justin-jackson-rb
samaje-perine
david-johnson-rb
ronald-jones-ii
khalil-herbert
kyle-juszczyk
jeremy-mcnichols
dontrell-hilliar

c:\Python\Python39\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Done for 2021
alvin-kamara
dalvin-cook
derrick-henry
david-montgomery
aaron-jones
jonathan-taylor
james-robinson
josh-jacobs
ezekiel-elliott
kareem-hunt
nick-chubb
mike-davis-rb
antonio-gibson
melvin-gordon
nyheim-hines
kenyan-drake
jd-mckissic
dandre-swift
chris-carson
ronald-jones-ii
david-johnson-rb
clyde-edwards-helaire
miles-sanders
jk-dobbins
chase-edmonds
austin-ekeler
james-conner
myles-gaskin
todd-gurley
giovani-bernard
devin-singletary
jeffery-wilson
wayne-gallman
latavius-murray
leonard-fournette
darrell-henderson
gus-edwards
jamaal-williams
jerick-mckinnon
adrian-peterson-min
tony-pollard
james-white
malcolm-brown
rex-burkhead
cam-akers
zack-moss
frank-gore
raheem-mostert
joe-mixon
boston-scott
brian-hill
kalen-ballage
damien-harris-rb
christian-mccaffrey
alexander-mattison
devontae-booker
carlos-hyde
duke-johnson
kyle-juszczyk
joshua-kelley
sony-michel
benjamin-snell-jr
leveon-bell
kerryon-johnson
salvon-ahmed
phillip-lindsay
samaje-perine
justin-jackson-rb
ty-johnson
jord

c:\Python\Python39\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Done for 2020
PCA Selected
Train dimensions (pre PCA):  (171, 26)
Test dimensions (pre PCA):  (95, 26)
Train dimensions (post PCA):  (171, 13)
Test dimensions (post PCA):  (95, 13)
Fitting 5 folds for each of 25 candidates, totalling 125 fits
[CV 1/5] END .......C=0.1, gamma=1, kernel=rbf;, score=-6.963 total time=   0.0s
[CV 2/5] END .......C=0.1, gamma=1, kernel=rbf;, score=-5.041 total time=   0.0s
[CV 3/5] END .......C=0.1, gamma=1, kernel=rbf;, score=-6.643 total time=   0.0s
[CV 4/5] END .......C=0.1, gamma=1, kernel=rbf;, score=-5.804 total time=   0.0s
[CV 5/5] END .......C=0.1, gamma=1, kernel=rbf;, score=-4.654 total time=   0.0s
[CV 1/5] END .....C=0.1, gamma=0.1, kernel=rbf;, score=-6.733 total time=   0.0s
[CV 2/5] END .....C=0.1, gamma=0.1, kernel=rbf;, score=-4.980 total time=   0.0s
[CV 3/5] END .....C=0.1, gamma=0.1, kernel=rbf;, score=-6.513 total time=   0.0s
[CV 4/5] END .....C=0.1, gamma=0.1, kernel=rbf;, score=-5.684 total time=   0.0s
[CV 5/5] END .....C=0.1, gam

,name,proj fpts,std fpts,class,2022 rank
0,christian-mccaffrey,16.343203,0.0,0,2
1,josh-jacobs,16.320048,0.0,0,3
2,austin-ekeler,15.736545,0.0,0,1
3,derrick-henry,14.628808,0.0,0,4
4,saquon-barkley,14.410446,0.0,0,5
5,nick-chubb,14.078572,0.0,1,6
6,joe-mixon,13.364743,0.0,0,10
7,dalvin-cook,12.906272,0.0,0,11
8,rhamondre-stevenson,12.880443,0.0,0,7
9,jamaal-williams,12.753420,0.0,1,13


In [3]:
# results['projections'].to_csv('2022_rb_projections.csv')

In [4]:
dists2022 = results['2022_df']
dists2021 = results['2021_df']
dists2020 = results['2020_df']
proj = xgb_2023('rb', 'PPR', dists2020, dists2021, dists2022)
proj.head(50)

======= Fold 0 ========
Our accuracy on the validation set is 3.855650862947407
======= Fold 1 ========
Our accuracy on the validation set is 3.942300925931987
======= Fold 2 ========
Our accuracy on the validation set is 3.9350781109602346
======= Fold 3 ========
Our accuracy on the validation set is 4.698276488058547
======= Fold 4 ========
Our accuracy on the validation set is 3.999253839457345
iteration 1: Dimensions = (95, 2)
iteration 2: Dimensions = (190, 2)
iteration 3: Dimensions = (285, 2)
iteration 4: Dimensions = (380, 2)
iteration 5: Dimensions = (475, 2)


,name,proj fpts,std fpts,class,2022 rank
0,josh-jacobs,16.563258,0.000183,0,3
1,austin-ekeler,15.942038,0.000373,0,1
2,christian-mccaffrey,15.864554,0.000267,0,2
3,derrick-henry,14.615619,0.000299,0,4
4,saquon-barkley,14.504797,0.000218,0,5
5,nick-chubb,13.965460,0.000103,1,6
6,joe-mixon,13.810614,0.000076,0,10
7,james-conner,13.233527,0.000126,2,18
8,dalvin-cook,13.033237,0.000170,0,11
9,dameon-pierce,12.598948,0.000234,1,26


In [5]:
# proj.to_csv('2022_rb_projections_xbr.csv')